In [1]:
import openrouteservice
import folium
import polyline
import pandas as pd

def plot_route_on_map(origin, destination, ev_stations, api_key_openrouteservice):

    client = openrouteservice.Client(key=api_key_openrouteservice)

    # Prepare coordinates
    waypoints = [origin]
    for stop in ev_stations:
        waypoints.append([stop['lon'], stop['lat']])
    waypoints.append(destination)
    coordinates = [waypoint for waypoint in waypoints]
    
    # Get the directions
    directions = client.directions(coordinates, profile='driving-car', units='km')
        
    # Extract the route coordinates from the response
    route_geometry_polyline = directions['routes'][0]['geometry']
    
    route_coordinates = polyline.decode(route_geometry_polyline)
    
    loc_origin_marker = [origin[1], origin[0]]
    loc_destination_marker = [destination[1], destination[0]]
    
    if route_coordinates:
        # Plotting the route on an actual map
        map_center = [(origin[1] + destination[1]) / 2, (origin[0] + destination[0]) / 2]
        map_obj = folium.Map(width=500, height=700, location=map_center, zoom_start=9)
        
        # Add markers for origin and destination
        folium.Marker(location=loc_origin_marker, popup='Origin', icon=folium.Icon(color='green',icon='home', prefix='fa')).add_to(map_obj)
        folium.Marker(location=loc_destination_marker, popup='Destination',icon=folium.Icon(color='blue',icon='location-pin', prefix='fa')).add_to(map_obj)
        
        # Add EV charging stations
        for stop in ev_stations:
            folium.Marker(
                location = [stop['lat'], stop['lon']],
                popup = stop['name'],
                icon = folium.Icon(color='red',icon='map-pin', prefix='fa')
            ).add_to(map_obj)
        
        # Add the route
        folium.PolyLine(locations=route_coordinates, color='blue').add_to(map_obj)

         # Extract segment information
        segments = directions['routes'][0]['segments']
        
        # Prepare data for the table
        table_data = []
        locations = ['Origin'] + [station['name'] for station in ev_stations] + ['Destination']
        
        cumulative_distance = 0
        cumulative_duration = 0
        
        for i, location in enumerate(locations):
            if i > 0:
                segment = segments[i-1]
                cumulative_distance += segment['distance']
                cumulative_duration += segment['duration'] / 60  # Convert to minutes
            
            table_data.append({
                'Point': location,
                'Cumulative Distance (km)': round(cumulative_distance, 2),
                'Cumulative Duration (mins)': round(cumulative_duration, 1)
            })
        
        # Create a pandas DataFrame
        df = pd.DataFrame(table_data)
        
        return map_obj, df
    else:
        print("Failed to retrieve the route.")
        return None

/Users/chanshutkeung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
api_key = "5b3ce3597851110001cf624842d6ce02dd3743baa1061059094207f5" # personal API key from OpenRouteService


# Example usage:
origin = [145.1551, -37.8508] # Melbourne CBD, VIC
destination = [144.3503, -38.1499] # Geelong, VIC
ev_stations = [
    {'name': 'Williamstown', 'lat': -37.8591, 'lon': 144.8944},
    {'name': 'Werribee Plaza', 'lat': -37.8871, 'lon': 144.6756},
    {'name': 'Little River', 'lat': -37.9941, 'lon': 144.5004}
]

route_map, route_data = plot_route_on_map(origin, destination, ev_stations, api_key)
print(route_data)
 
route_map


            Point  Cumulative Distance (km)  Cumulative Duration (mins)
0          Origin                      0.00                         0.0
1    Williamstown                     30.71                        43.4
2  Werribee Plaza                     53.36                        71.9
3    Little River                     76.41                        96.4
4     Destination                    107.96                       127.2
